In [107]:
import pandas as pd
import datetime

In [108]:
df = pd.read_csv("trans.csv")

In [109]:
df

,transaction_name,amount,date
0,bike servicing,-2500,2020-05-13 15:25:10+00:00
1,vegetable bought with coke,-160,2020-05-16 14:15:16+00:00
2,sold cycle,2800,2020-05-17 11:23:11+00:00
3,purchase a laptop,-50000,2020-06-12 18:03:04.172291+00:00
4,income from bank,200,2020-06-12 18:57:36.990277+00:00
5,brought purse,-250,2020-06-12 19:00:19.624901+00:00
6,monthly income,80000,2020-06-12 19:29:51.126575+00:00
7,Give money to bibek,-1000,2020-06-12 19:34:25.978902+00:00
8,Bag changed,-1200,2020-06-13 17:05:51.360723+00:00
9,shopping bhatbhateini,-1500,2020-06-14 05:55:52.589900+00:00


In [110]:
df.drop("transaction_name",axis=1,inplace = True)

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
amount    15 non-null int64
date      15 non-null object
dtypes: int64(1), object(1)
memory usage: 368.0+ bytes


In [112]:
df['date'] = pd.to_datetime(df['date']).dt.date


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
amount    15 non-null int64
date      15 non-null object
dtypes: int64(1), object(1)
memory usage: 368.0+ bytes


In [114]:

df['date'] = pd.to_datetime(df['date'])

In [115]:
df

,amount,date
0,-2500,2020-05-13
1,-160,2020-05-16
2,2800,2020-05-17
3,-50000,2020-06-12
4,200,2020-06-12
5,-250,2020-06-12
6,80000,2020-06-12
7,-1000,2020-06-12
8,-1200,2020-06-13
9,-1500,2020-06-14


In [116]:
#lets combine the amount by date
dfcopy = df.groupby('date').sum()
dfcopy

,amount
date,
2020-05-13,-2500
2020-05-16,-160
2020-05-17,2800
2020-06-12,28950
2020-06-13,-1200
2020-06-14,-1500
2020-06-16,-2000
2020-06-17,1100
2020-06-19,-1200


In [117]:
dfcopy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10 entries, 2020-05-13 to 2020-06-20
Data columns (total 1 columns):
amount    10 non-null int64
dtypes: int64(1)
memory usage: 160.0 bytes


In [118]:
# working with days only
dfcopy['days_from_start'] = (dfcopy.index - dfcopy.index[0]).days
dfcopy

,amount,days_from_start
date,,
2020-05-13,-2500,0
2020-05-16,-160,3
2020-05-17,2800,4
2020-06-12,28950,30
2020-06-13,-1200,31
2020-06-14,-1500,32
2020-06-16,-2000,34
2020-06-17,1100,35
2020-06-19,-1200,37


In [119]:
remaining = []
eachamount = dfcopy.amount[0]
remaining.append(dfcopy.amount[0])

for i in range(len(dfcopy)-1):
    eachamount += dfcopy.amount[i+1]
    remaining.append(eachamount)


In [120]:
X = dfcopy["days_from_start"].values
Y= remaining

In [124]:
#lets reshapre the input
X = X.reshape([-1,1])

In [126]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [149]:
model.score(X,Y)


0.9179841711152297

In [128]:
ypredicts = model.predict(X)

In [129]:
Y,ypredicts

([-2500, -2660, 140, 29090, 27890, 26390, 24390, 25490, 24290, 23090],
 array([-2715.89471726,  -222.83389137,   608.18638393, 22214.71354167,
        23045.73381696, 23876.75409226, 25538.79464286, 26369.81491815,
        28031.85546875, 28862.87574405]))

In [130]:
from sklearn.metrics import mean_absolute_error,r2_score
print(mean_absolute_error(Y,ypredicts))
print(r2_score(Y,ypredicts))

2889.738653273809
0.9179841711152297


## Futute 7 days prediction from last transaction

In [136]:
endindex = len(dfcopy)-1
days_from_start = dfcopy['days_from_start'][endindex]
days_from_start

38

In [140]:
from datetime import datetime,timedelta
nextdate = dfcopy.index[endindex]
nextdatetime = []
nextdatedays = []
for i in range(1,7):
    nextdatedays.append(days_from_start + i)
    nextdate += timedelta(days=1)
    nextdatetime.append(nextdate)

In [156]:
dfpredict = pd.DataFrame({
    'days':nextdatedays,
})

In [157]:
yfuture = model.predict(dfpredict)

In [158]:
dfpredict["result"] = pd.DataFrame(yfuture)
dfpredict["date"] = pd.DataFrame(nextdatetime)

In [159]:
remaining

[-2500, -2660, 140, 29090, 27890, 26390, 24390, 25490, 24290, 23090]

In [160]:
dfpredict

,days,result,date
0,39,29693.896019,2020-06-21
1,40,30524.916295,2020-06-22
2,41,31355.936570,2020-06-23
3,42,32186.956845,2020-06-24
4,43,33017.977121,2020-06-25
5,44,33848.997396,2020-06-26


23090